In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)
    
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)

#     c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
#     c3 = BatchNormalization(axis=1)(c3)
#     c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
#     c3 = BatchNormalization(axis=1)(c3)
#     p3 = MaxPooling2D((2, 2)) (c3)

#     c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
#     c4 = BatchNormalization(axis=1)(c4)
#     c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
#     c4 = BatchNormalization(axis=1)(c4)
#     p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

#     c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
#     c5 = BatchNormalization(axis=1)(c5)
#     c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
#     c5 = BatchNormalization(axis=1)(c5)

#     u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
#     u6 = concatenate([u6, c4])
#     c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
#     c6 = BatchNormalization(axis=1)(c6)
#     c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
#     c6 = BatchNormalization(axis=1)(c6)

#     u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
#     u7 = concatenate([u7, c3])
#     c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
#     c7 = BatchNormalization(axis=1)(c7)
#     c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
#     c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c3)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/depthtest2/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, callbacks=[<keras.ca..., validation_data=(array([[[..., verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 13s 127ms/step - loss: 5.0751 - binary_crossentropy: 0.4207 - jaccard_coef_int: 0.1286 - binary_accuracy: 0.8274 - val_loss: 7.2870 - val_binary_crossentropy: 0.8418 - val_jaccard_coef_int: 0.0093 - val_binary_accuracy: 0.7899
Epoch 2/50
100/100 [==============================] - 8s 83ms/step - loss: 4.5141 - binary_crossentropy: 0.5636 - jaccard_coef_int: 0.2235 - binary_accuracy: 0.8245 - val_loss: 8.3437 - val_binary_crossentropy: 1.0292 - val_jaccard_coef_int: 8.9892e-16 - val_binary_accuracy: 0.8009
Epoch 3/50
100/100 [==============================] - 8s 80ms/step - loss: 4.2062 - binary_crossentropy: 0.6576 - jaccard_coef_int: 0.2255 - binary_accuracy: 0.8369 - val_loss: 4.1314 - val_binary_crossentropy: 0.8022 - val_jaccard_coef_int: 0.3616 - val_binary_accuracy: 0.8103
Epoch 4/50
100/100 [==============================] - 8s 82ms/step - loss: 3.9102 - binary_crossentropy: 0.7251 - jaccard_coef_int: 0.2776 - binary_accuracy:

Epoch 7/50
100/100 [==============================] - 10s 102ms/step - loss: 2.5471 - binary_crossentropy: 0.0724 - jaccard_coef_int: 0.1108 - binary_accuracy: 0.9883 - val_loss: 3.9079 - val_binary_crossentropy: 0.0848 - val_jaccard_coef_int: 0.0252 - val_binary_accuracy: 0.9879
Epoch 8/50
100/100 [==============================] - 10s 101ms/step - loss: 2.3345 - binary_crossentropy: 0.0689 - jaccard_coef_int: 0.1270 - binary_accuracy: 0.9902 - val_loss: 4.6166 - val_binary_crossentropy: 0.0912 - val_jaccard_coef_int: 0.0109 - val_binary_accuracy: 0.9879
Epoch 9/50
100/100 [==============================] - 10s 104ms/step - loss: 2.8464 - binary_crossentropy: 0.0937 - jaccard_coef_int: 0.0869 - binary_accuracy: 0.9858 - val_loss: 3.9075 - val_binary_crossentropy: 0.3328 - val_jaccard_coef_int: 0.0462 - val_binary_accuracy: 0.9502
Epoch 10/50
100/100 [==============================] - 10s 102ms/step - loss: 2.6087 - binary_crossentropy: 0.0929 - jaccard_coef_int: 0.1042 - binary_accura

100/100 [==============================] - 10s 103ms/step - loss: 2.1837 - binary_crossentropy: 0.0920 - jaccard_coef_int: 0.1549 - binary_accuracy: 0.9913 - val_loss: 2.6635 - val_binary_crossentropy: 0.1958 - val_jaccard_coef_int: 0.0930 - val_binary_accuracy: 0.9823
Epoch 37/50
100/100 [==============================] - 10s 103ms/step - loss: 2.0396 - binary_crossentropy: 0.0936 - jaccard_coef_int: 0.1605 - binary_accuracy: 0.9915 - val_loss: 2.9431 - val_binary_crossentropy: 0.2883 - val_jaccard_coef_int: 0.0789 - val_binary_accuracy: 0.9748
Epoch 38/50
100/100 [==============================] - 10s 102ms/step - loss: 1.9590 - binary_crossentropy: 0.0962 - jaccard_coef_int: 0.1741 - binary_accuracy: 0.9915 - val_loss: 2.7524 - val_binary_crossentropy: 0.2524 - val_jaccard_coef_int: 0.0901 - val_binary_accuracy: 0.9787
Epoch 39/50
100/100 [==============================] - 10s 103ms/step - loss: 1.9990 - binary_crossentropy: 0.0954 - jaccard_coef_int: 0.1645 - binary_accuracy: 0.991

...

Epoch 11/50
100/100 [==============================] - 8s 84ms/step - loss: 2.1827 - binary_crossentropy: 0.1868 - jaccard_coef_int: 0.1871 - binary_accuracy: 0.9709 - val_loss: 2.8431 - val_binary_crossentropy: 0.1130 - val_jaccard_coef_int: 0.0747 - val_binary_accuracy: 0.9837
Epoch 12/50
100/100 [==============================] - 8s 83ms/step - loss: 2.1902 - binary_crossentropy: 0.1948 - jaccard_coef_int: 0.1869 - binary_accuracy: 0.9705 - val_loss: 2.4519 - val_binary_crossentropy: 0.1076 - val_jaccard_coef_int: 0.1140 - val_binary_accuracy: 0.9838
Epoch 13/50
100/100 [==============================] - 8s 85ms/step - loss: 2.1092 - binary_crossentropy: 0.1943 - jaccard_coef_int: 0.1975 - binary_accuracy: 0.9717 - val_loss: 2.6585 - val_binary_crossentropy: 0.1032 - val_jaccard_coef_int: 0.0885 - val_binary_accuracy: 0.9869
Epoch 14/50
100/100 [==============================] - 8s 83ms/step - loss: 2.0641 - binary_crossentropy: 0.1984 - jaccard_coef_int: 0.2044 - binary_accuracy: 0

Epoch 17/50
100/100 [==============================] - 11s 108ms/step - loss: 2.5800 - binary_crossentropy: 0.2737 - jaccard_coef_int: 0.1236 - binary_accuracy: 0.9706 - val_loss: 3.8729 - val_binary_crossentropy: 0.2034 - val_jaccard_coef_int: 0.0296 - val_binary_accuracy: 0.9776
Epoch 18/50
100/100 [==============================] - 11s 109ms/step - loss: 2.5284 - binary_crossentropy: 0.2690 - jaccard_coef_int: 0.1278 - binary_accuracy: 0.9716 - val_loss: 3.6220 - val_binary_crossentropy: 0.1955 - val_jaccard_coef_int: 0.0373 - val_binary_accuracy: 0.9793
Epoch 19/50
100/100 [==============================] - 11s 110ms/step - loss: 2.5117 - binary_crossentropy: 0.2808 - jaccard_coef_int: 0.1316 - binary_accuracy: 0.9707 - val_loss: 3.7227 - val_binary_crossentropy: 0.4013 - val_jaccard_coef_int: 0.0469 - val_binary_accuracy: 0.9588
Epoch 20/50
100/100 [==============================] - 11s 109ms/step - loss: 2.4412 - binary_crossentropy: 0.2709 - jaccard_coef_int: 0.1383 - binary_acc

Epoch 23/50
100/100 [==============================] - 11s 108ms/step - loss: 3.2920 - binary_crossentropy: 0.7491 - jaccard_coef_int: 0.1323 - binary_accuracy: 0.9217 - val_loss: 4.1090 - val_binary_crossentropy: 0.9655 - val_jaccard_coef_int: 0.0843 - val_binary_accuracy: 0.8982
Epoch 24/50
100/100 [==============================] - 11s 108ms/step - loss: 3.2495 - binary_crossentropy: 0.7375 - jaccard_coef_int: 0.1332 - binary_accuracy: 0.9239 - val_loss: 4.2953 - val_binary_crossentropy: 0.6723 - val_jaccard_coef_int: 0.0439 - val_binary_accuracy: 0.9275
Epoch 25/50
100/100 [==============================] - 11s 108ms/step - loss: 3.2280 - binary_crossentropy: 0.7415 - jaccard_coef_int: 0.1359 - binary_accuracy: 0.9240 - val_loss: 6.4895 - val_binary_crossentropy: 2.3732 - val_jaccard_coef_int: 0.0750 - val_binary_accuracy: 0.7584
Epoch 26/50
100/100 [==============================] - 11s 108ms/step - loss: 3.2026 - binary_crossentropy: 0.7576 - jaccard_coef_int: 0.1406 - binary_acc

Epoch 1/50
100/100 [==============================] - 25s 253ms/step - loss: 5.2170 - binary_crossentropy: 0.2410 - jaccard_coef_int: 0.0011 - binary_accuracy: 0.9362 - val_loss: 4.7843 - val_binary_crossentropy: 0.2606 - val_jaccard_coef_int: 0.0010 - val_binary_accuracy: 0.9341
Epoch 2/50
100/100 [==============================] - 8s 82ms/step - loss: 4.7270 - binary_crossentropy: 0.2597 - jaccard_coef_int: 0.0226 - binary_accuracy: 0.9313 - val_loss: 6.3866 - val_binary_crossentropy: 0.3478 - val_jaccard_coef_int: 2.7904e-15 - val_binary_accuracy: 0.9346
Epoch 3/50
100/100 [==============================] - 8s 85ms/step - loss: 4.3535 - binary_crossentropy: 0.3327 - jaccard_coef_int: 0.0529 - binary_accuracy: 0.9181 - val_loss: 7.1993 - val_binary_crossentropy: 0.4050 - val_jaccard_coef_int: 2.7896e-15 - val_binary_accuracy: 0.9346
Epoch 4/50
100/100 [==============================] - 8s 82ms/step - loss: 4.1786 - binary_crossentropy: 0.3993 - jaccard_coef_int: 0.0622 - binary_accur

Epoch 7/50
100/100 [==============================] - 11s 107ms/step - loss: 2.8597 - binary_crossentropy: 0.0813 - jaccard_coef_int: 0.1335 - binary_accuracy: 0.9850 - val_loss: 9.8454 - val_binary_crossentropy: 0.0057 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 8/50
100/100 [==============================] - 11s 109ms/step - loss: 2.8204 - binary_crossentropy: 0.0821 - jaccard_coef_int: 0.1468 - binary_accuracy: 0.9851 - val_loss: 5.1127 - val_binary_crossentropy: 0.0052 - val_jaccard_coef_int: 0.6313 - val_binary_accuracy: 0.9994
Epoch 9/50
100/100 [==============================] - 11s 106ms/step - loss: 2.9047 - binary_crossentropy: 0.0785 - jaccard_coef_int: 0.1251 - binary_accuracy: 0.9867 - val_loss: 6.8835 - val_binary_crossentropy: 0.0597 - val_jaccard_coef_int: 0.0136 - val_binary_accuracy: 0.9880
Epoch 10/50
100/100 [==============================] - 10s 105ms/step - loss: 2.8256 - binary_crossentropy: 0.0860 - jaccard_coef_int: 0.1374 - binary_accura

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest2/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, callbacks=[<keras.ca..., validation_data=(array([[[..., verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 29s 293ms/step - loss: 5.8292 - binary_crossentropy: 0.6527 - jaccard_coef_int: 0.2609 - binary_accuracy: 0.7559 - val_loss: 5.3211 - val_binary_crossentropy: 1.0922 - val_jaccard_coef_int: 0.1183 - val_binary_accuracy: 0.8013
Epoch 2/50
100/100 [==============================] - 11s 110ms/step - loss: 4.6046 - binary_crossentropy: 0.8598 - jaccard_coef_int: 0.3760 - binary_accuracy: 0.7810 - val_loss: 4.6095 - val_binary_crossentropy: 0.9920 - val_jaccard_coef_int: 0.2556 - val_binary_accuracy: 0.7996
Epoch 3/50
100/100 [==============================] - 11s 110ms/step - loss: 4.5082 - binary_crossentropy: 0.9682 - jaccard_coef_int: 0.3792 - binary_accuracy: 0.7833 - val_loss: 4.4396 - val_binary_crossentropy: 0.9256 - val_jaccard_coef_int: 0.3523 - val_binary_accuracy: 0.7918
Epoch 4/50
100/100 [==============================] - 11s 108ms/step - loss: 4.4029 - binary_crossentropy: 1.0515 - jaccard_coef_int: 0.3871 - binary_accurac

Epoch 7/50
100/100 [==============================] - 11s 107ms/step - loss: 4.0066 - binary_crossentropy: 0.7765 - jaccard_coef_int: 0.1346 - binary_accuracy: 0.8787 - val_loss: 6.2868 - val_binary_crossentropy: 0.7927 - val_jaccard_coef_int: 0.0104 - val_binary_accuracy: 0.8850
Epoch 8/50
100/100 [==============================] - 11s 106ms/step - loss: 3.9421 - binary_crossentropy: 0.7900 - jaccard_coef_int: 0.1337 - binary_accuracy: 0.8824 - val_loss: 6.0866 - val_binary_crossentropy: 0.8282 - val_jaccard_coef_int: 0.0135 - val_binary_accuracy: 0.8838
Epoch 9/50
100/100 [==============================] - 11s 105ms/step - loss: 3.9018 - binary_crossentropy: 0.8212 - jaccard_coef_int: 0.1368 - binary_accuracy: 0.8834 - val_loss: 5.1258 - val_binary_crossentropy: 0.9280 - val_jaccard_coef_int: 0.0462 - val_binary_accuracy: 0.8713
Epoch 10/50
100/100 [==============================] - 11s 108ms/step - loss: 3.8843 - binary_crossentropy: 0.8574 - jaccard_coef_int: 0.1379 - binary_accura

Epoch 13/50
100/100 [==============================] - 8s 81ms/step - loss: 2.4170 - binary_crossentropy: 0.1213 - jaccard_coef_int: 0.1262 - binary_accuracy: 0.9819 - val_loss: 2.9621 - val_binary_crossentropy: 0.2808 - val_jaccard_coef_int: 0.1050 - val_binary_accuracy: 0.9563
Epoch 14/50
100/100 [==============================] - 8s 82ms/step - loss: 2.4109 - binary_crossentropy: 0.1272 - jaccard_coef_int: 0.1273 - binary_accuracy: 0.9815 - val_loss: 5.2210 - val_binary_crossentropy: 0.1567 - val_jaccard_coef_int: 0.0072 - val_binary_accuracy: 0.9818
Epoch 15/50
100/100 [==============================] - 8s 81ms/step - loss: 2.3911 - binary_crossentropy: 0.1279 - jaccard_coef_int: 0.1283 - binary_accuracy: 0.9822 - val_loss: 3.4465 - val_binary_crossentropy: 0.4963 - val_jaccard_coef_int: 0.0878 - val_binary_accuracy: 0.9370
Epoch 16/50
100/100 [==============================] - 8s 84ms/step - loss: 2.3606 - binary_crossentropy: 0.1347 - jaccard_coef_int: 0.1333 - binary_accuracy: 0

...

100/100 [==============================] - 8s 84ms/step - loss: 2.2122 - binary_crossentropy: 0.3514 - jaccard_coef_int: 0.1996 - binary_accuracy: 0.9633 - val_loss: 3.0289 - val_binary_crossentropy: 0.3458 - val_jaccard_coef_int: 0.0835 - val_binary_accuracy: 0.9662
Epoch 26/50
100/100 [==============================] - 8s 84ms/step - loss: 2.1969 - binary_crossentropy: 0.3463 - jaccard_coef_int: 0.1991 - binary_accuracy: 0.9643 - val_loss: 2.6627 - val_binary_crossentropy: 0.4216 - val_jaccard_coef_int: 0.1347 - val_binary_accuracy: 0.9590
Epoch 27/50
100/100 [==============================] - 8s 83ms/step - loss: 2.1478 - binary_crossentropy: 0.3495 - jaccard_coef_int: 0.2091 - binary_accuracy: 0.9643 - val_loss: 2.7573 - val_binary_crossentropy: 0.4414 - val_jaccard_coef_int: 0.1266 - val_binary_accuracy: 0.9571
Epoch 28/50
100/100 [==============================] - 8s 83ms/step - loss: 2.1497 - binary_crossentropy: 0.3493 - jaccard_coef_int: 0.2069 - binary_accuracy: 0.9649 - val_

Epoch 2/50
100/100 [==============================] - 8s 83ms/step - loss: 2.5253 - binary_crossentropy: 0.1538 - jaccard_coef_int: 0.1835 - binary_accuracy: 0.9610 - val_loss: 2.7995 - val_binary_crossentropy: 0.1633 - val_jaccard_coef_int: 0.1409 - val_binary_accuracy: 0.9578
Epoch 3/50
100/100 [==============================] - 8s 85ms/step - loss: 2.3420 - binary_crossentropy: 0.1738 - jaccard_coef_int: 0.1985 - binary_accuracy: 0.9617 - val_loss: 2.7229 - val_binary_crossentropy: 0.1744 - val_jaccard_coef_int: 0.1471 - val_binary_accuracy: 0.9614
Epoch 4/50
100/100 [==============================] - 8s 83ms/step - loss: 2.3106 - binary_crossentropy: 0.1949 - jaccard_coef_int: 0.1965 - binary_accuracy: 0.9620 - val_loss: 2.5283 - val_binary_crossentropy: 0.1911 - val_jaccard_coef_int: 0.1496 - val_binary_accuracy: 0.9626
Epoch 5/50
100/100 [==============================] - 8s 85ms/step - loss: 2.2232 - binary_crossentropy: 0.2105 - jaccard_coef_int: 0.2114 - binary_accuracy: 0.961

Epoch 8/50
100/100 [==============================] - 8s 83ms/step - loss: 2.9044 - binary_crossentropy: 0.8218 - jaccard_coef_int: 0.6131 - binary_accuracy: 0.8508 - val_loss: 3.0042 - val_binary_crossentropy: 0.9035 - val_jaccard_coef_int: 0.5914 - val_binary_accuracy: 0.8462
Epoch 9/50
100/100 [==============================] - 8s 85ms/step - loss: 2.9000 - binary_crossentropy: 0.8518 - jaccard_coef_int: 0.6120 - binary_accuracy: 0.8509 - val_loss: 2.7907 - val_binary_crossentropy: 0.8854 - val_jaccard_coef_int: 0.6329 - val_binary_accuracy: 0.8551
Epoch 10/50
100/100 [==============================] - 8s 83ms/step - loss: 2.8809 - binary_crossentropy: 0.8805 - jaccard_coef_int: 0.6173 - binary_accuracy: 0.8514 - val_loss: 2.8109 - val_binary_crossentropy: 0.9016 - val_jaccard_coef_int: 0.6491 - val_binary_accuracy: 0.8524
Epoch 11/50
100/100 [==============================] - 8s 83ms/step - loss: 2.8884 - binary_crossentropy: 0.9104 - jaccard_coef_int: 0.6217 - binary_accuracy: 0.8

Epoch 14/50
100/100 [==============================] - 8s 83ms/step - loss: 0.6879 - binary_crossentropy: 0.0575 - jaccard_coef_int: 0.6340 - binary_accuracy: 0.9930 - val_loss: 1.0872 - val_binary_crossentropy: 0.1263 - val_jaccard_coef_int: 0.4247 - val_binary_accuracy: 0.9860
Epoch 15/50
100/100 [==============================] - 9s 86ms/step - loss: 0.6197 - binary_crossentropy: 0.0556 - jaccard_coef_int: 0.6484 - binary_accuracy: 0.9931 - val_loss: 1.0914 - val_binary_crossentropy: 0.1451 - val_jaccard_coef_int: 0.4261 - val_binary_accuracy: 0.9857
Epoch 16/50
100/100 [==============================] - 9s 86ms/step - loss: 0.5927 - binary_crossentropy: 0.0535 - jaccard_coef_int: 0.6582 - binary_accuracy: 0.9935 - val_loss: 1.0833 - val_binary_crossentropy: 0.1364 - val_jaccard_coef_int: 0.4270 - val_binary_accuracy: 0.9860
Epoch 17/50
100/100 [==============================] - 8s 84ms/step - loss: 0.5877 - binary_crossentropy: 0.0603 - jaccard_coef_int: 0.6557 - binary_accuracy: 0

Epoch 20/50
100/100 [==============================] - 11s 109ms/step - loss: 4.2803 - binary_crossentropy: 0.1313 - jaccard_coef_int: 0.0358 - binary_accuracy: 0.9754 - val_loss: 3.2408 - val_binary_crossentropy: 0.2164 - val_jaccard_coef_int: 0.0676 - val_binary_accuracy: 0.9720
Epoch 21/50
100/100 [==============================] - 11s 110ms/step - loss: 4.4787 - binary_crossentropy: 0.2530 - jaccard_coef_int: 0.0302 - binary_accuracy: 0.9729 - val_loss: 3.1857 - val_binary_crossentropy: 0.2879 - val_jaccard_coef_int: 0.0788 - val_binary_accuracy: 0.9685
Epoch 22/50
100/100 [==============================] - 11s 109ms/step - loss: 3.9208 - binary_crossentropy: 0.1119 - jaccard_coef_int: 0.0407 - binary_accuracy: 0.9789 - val_loss: 3.5531 - val_binary_crossentropy: 0.1251 - val_jaccard_coef_int: 0.0531 - val_binary_accuracy: 0.9739
Epoch 23/50
100/100 [==============================] - 11s 108ms/step - loss: 3.8067 - binary_crossentropy: 0.1089 - jaccard_coef_int: 0.0433 - binary_acc

In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest2/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, epochs=50, validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 13s 128ms/step - loss: 2.0404 - binary_crossentropy: 0.0493 - jaccard_coef_int: 0.6512 - binary_accuracy: 0.9907 - val_loss: 1.0422 - val_binary_crossentropy: 0.0493 - val_jaccard_coef_int: 0.6476 - val_binary_accuracy: 0.9904
Epoch 2/50
100/100 [==============================] - 8s 80ms/step - loss: 0.7838 - binary_crossentropy: 0.0236 - jaccard_coef_int: 0.7347 - binary_accuracy: 0.9953 - val_loss: 0.6602 - val_binary_crossentropy: 0.0478 - val_jaccard_coef_int: 0.6465 - val_binary_accuracy: 0.9908
Epoch 3/50
100/100 [==============================] - 9s 85ms/step - loss: 0.9834 - binary_crossentropy: 0.0548 - jaccard_coef_int: 0.7399 - binary_accuracy: 0.9865 - val_loss: 0.5966 - val_binary_crossentropy: 0.0542 - val_jaccard_coef_int: 0.6461 - val_binary_accuracy: 0.9908
Epoch 4/50
100/100 [==============================] - 8s 83ms/step - loss: 0.5295 - binary_crossentropy: 0.0221 - jaccard_coef_int: 0.7577 - binary_accuracy: 0.9

100/100 [==============================] - 8s 80ms/step - loss: 0.3653 - binary_crossentropy: 0.0410 - jaccard_coef_int: 0.7549 - binary_accuracy: 0.9957 - val_loss: 0.5586 - val_binary_crossentropy: 0.0951 - val_jaccard_coef_int: 0.6606 - val_binary_accuracy: 0.9911
Epoch 31/50
100/100 [==============================] - 8s 82ms/step - loss: 0.3333 - binary_crossentropy: 0.0414 - jaccard_coef_int: 0.7752 - binary_accuracy: 0.9956 - val_loss: 0.5532 - val_binary_crossentropy: 0.0929 - val_jaccard_coef_int: 0.6644 - val_binary_accuracy: 0.9912
Epoch 32/50
100/100 [==============================] - 8s 80ms/step - loss: 0.3393 - binary_crossentropy: 0.0450 - jaccard_coef_int: 0.7713 - binary_accuracy: 0.9955 - val_loss: 0.5573 - val_binary_crossentropy: 0.0986 - val_jaccard_coef_int: 0.6619 - val_binary_accuracy: 0.9910
Epoch 33/50
100/100 [==============================] - 8s 82ms/step - loss: 0.3557 - binary_crossentropy: 0.0445 - jaccard_coef_int: 0.7609 - binary_accuracy: 0.9954 - val_

Epoch 7/50
100/100 [==============================] - 11s 107ms/step - loss: 4.6153 - binary_crossentropy: 1.1967 - jaccard_coef_int: 0.2959 - binary_accuracy: 0.7892 - val_loss: 4.7178 - val_binary_crossentropy: 1.1202 - val_jaccard_coef_int: 0.2178 - val_binary_accuracy: 0.8016
Epoch 8/50
100/100 [==============================] - 11s 107ms/step - loss: 4.6501 - binary_crossentropy: 1.2667 - jaccard_coef_int: 0.2969 - binary_accuracy: 0.7867 - val_loss: 4.6849 - val_binary_crossentropy: 1.4169 - val_jaccard_coef_int: 0.2847 - val_binary_accuracy: 0.7867
Epoch 9/50
100/100 [==============================] - 11s 107ms/step - loss: 4.6117 - binary_crossentropy: 1.3096 - jaccard_coef_int: 0.2983 - binary_accuracy: 0.7885 - val_loss: 4.5702 - val_binary_crossentropy: 1.1420 - val_jaccard_coef_int: 0.2750 - val_binary_accuracy: 0.7959
Epoch 10/50
100/100 [==============================] - 11s 107ms/step - loss: 4.5615 - binary_crossentropy: 1.3439 - jaccard_coef_int: 0.3018 - binary_accura

...

100/100 [==============================] - 8s 82ms/step - loss: 2.4164 - binary_crossentropy: 0.7017 - jaccard_coef_int: 0.2753 - binary_accuracy: 0.9302 - val_loss: 3.3041 - val_binary_crossentropy: 0.9968 - val_jaccard_coef_int: 0.1860 - val_binary_accuracy: 0.8987
Epoch 34/50
100/100 [==============================] - 8s 83ms/step - loss: 2.4107 - binary_crossentropy: 0.7045 - jaccard_coef_int: 0.2742 - binary_accuracy: 0.9308 - val_loss: 3.3328 - val_binary_crossentropy: 1.0436 - val_jaccard_coef_int: 0.1812 - val_binary_accuracy: 0.8986
Epoch 35/50
100/100 [==============================] - 8s 82ms/step - loss: 2.3759 - binary_crossentropy: 0.6996 - jaccard_coef_int: 0.2788 - binary_accuracy: 0.9320 - val_loss: 3.2688 - val_binary_crossentropy: 0.9344 - val_jaccard_coef_int: 0.1570 - val_binary_accuracy: 0.9113
Epoch 36/50
100/100 [==============================] - 8s 81ms/step - loss: 2.3650 - binary_crossentropy: 0.7045 - jaccard_coef_int: 0.2807 - binary_accuracy: 0.9321 - val_

100/100 [==============================] - 8s 83ms/step - loss: 2.5378 - binary_crossentropy: 0.0600 - jaccard_coef_int: 0.2101 - binary_accuracy: 0.9903 - val_loss: 42.3523 - val_binary_crossentropy: 1.9196 - val_jaccard_coef_int: 4.2442e-16 - val_binary_accuracy: 0.5656
Epoch 11/50
100/100 [==============================] - 8s 82ms/step - loss: 2.6555 - binary_crossentropy: 0.0751 - jaccard_coef_int: 0.1822 - binary_accuracy: 0.9873 - val_loss: 28.0455 - val_binary_crossentropy: 0.0033 - val_jaccard_coef_int: 0.6125 - val_binary_accuracy: 0.9990
Epoch 12/50
100/100 [==============================] - 8s 82ms/step - loss: 2.2383 - binary_crossentropy: 0.0698 - jaccard_coef_int: 0.2157 - binary_accuracy: 0.9898 - val_loss: 29.7495 - val_binary_crossentropy: 0.0062 - val_jaccard_coef_int: 0.4750 - val_binary_accuracy: 0.9984
Epoch 13/50
100/100 [==============================] - 8s 83ms/step - loss: 2.3750 - binary_crossentropy: 0.0733 - jaccard_coef_int: 0.2143 - binary_accuracy: 0.9895

100/100 [==============================] - 8s 83ms/step - loss: 2.0610 - binary_crossentropy: 0.0923 - jaccard_coef_int: 0.2506 - binary_accuracy: 0.9903 - val_loss: 28.6393 - val_binary_crossentropy: 0.0318 - val_jaccard_coef_int: 0.0750 - val_binary_accuracy: 0.9949
Epoch 40/50
100/100 [==============================] - 8s 82ms/step - loss: 2.1119 - binary_crossentropy: 0.0998 - jaccard_coef_int: 0.2150 - binary_accuracy: 0.9894 - val_loss: 27.1667 - val_binary_crossentropy: 0.0075 - val_jaccard_coef_int: 0.4750 - val_binary_accuracy: 0.9987
Epoch 41/50
100/100 [==============================] - 8s 81ms/step - loss: 1.9355 - binary_crossentropy: 0.0912 - jaccard_coef_int: 0.2545 - binary_accuracy: 0.9908 - val_loss: 25.7246 - val_binary_crossentropy: 0.0020 - val_jaccard_coef_int: 0.7625 - val_binary_accuracy: 0.9996
Epoch 42/50
100/100 [==============================] - 8s 83ms/step - loss: 1.7712 - binary_crossentropy: 0.0882 - jaccard_coef_int: 0.2718 - binary_accuracy: 0.9913 - v

Epoch 16/50
100/100 [==============================] - 8s 82ms/step - loss: 2.1500 - binary_crossentropy: 0.0597 - jaccard_coef_int: 0.1501 - binary_accuracy: 0.9938 - val_loss: 5.3514 - val_binary_crossentropy: 0.0330 - val_jaccard_coef_int: 0.0050 - val_binary_accuracy: 0.9960
Epoch 17/50
100/100 [==============================] - 8s 82ms/step - loss: 2.3165 - binary_crossentropy: 0.0637 - jaccard_coef_int: 0.1442 - binary_accuracy: 0.9933 - val_loss: 4.3206 - val_binary_crossentropy: 0.1118 - val_jaccard_coef_int: 0.0161 - val_binary_accuracy: 0.9881
Epoch 18/50
100/100 [==============================] - 8s 82ms/step - loss: 2.2012 - binary_crossentropy: 0.0629 - jaccard_coef_int: 0.1529 - binary_accuracy: 0.9937 - val_loss: 5.0467 - val_binary_crossentropy: 0.1708 - val_jaccard_coef_int: 0.0086 - val_binary_accuracy: 0.9833
Epoch 19/50
100/100 [==============================] - 8s 83ms/step - loss: 1.9908 - binary_crossentropy: 0.0543 - jaccard_coef_int: 0.1725 - binary_accuracy: 0

100/100 [==============================] - 8s 83ms/step - loss: 1.7038 - binary_crossentropy: 0.0599 - jaccard_coef_int: 0.2195 - binary_accuracy: 0.9951 - val_loss: 12.1280 - val_binary_crossentropy: 0.0281 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 46/50
100/100 [==============================] - 8s 84ms/step - loss: 1.6557 - binary_crossentropy: 0.0588 - jaccard_coef_int: 0.2312 - binary_accuracy: 0.9952 - val_loss: 11.9357 - val_binary_crossentropy: 0.0347 - val_jaccard_coef_int: 0.0250 - val_binary_accuracy: 0.9969
Epoch 47/50
100/100 [==============================] - 8s 82ms/step - loss: 1.6518 - binary_crossentropy: 0.0559 - jaccard_coef_int: 0.2356 - binary_accuracy: 0.9955 - val_loss: 5.6894 - val_binary_crossentropy: 0.4549 - val_jaccard_coef_int: 0.0061 - val_binary_accuracy: 0.9626
Epoch 48/50
100/100 [==============================] - 8s 83ms/step - loss: 1.5804 - binary_crossentropy: 0.0580 - jaccard_coef_int: 0.2386 - binary_accuracy: 0.9953 - va

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest2/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., steps_per_epoch=100, epochs=50, validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 17s 172ms/step - loss: 2.8996 - binary_crossentropy: 0.2907 - jaccard_coef_int: 0.3904 - binary_accuracy: 0.9072 - val_loss: 2.5378 - val_binary_crossentropy: 0.2125 - val_jaccard_coef_int: 0.4464 - val_binary_accuracy: 0.9283
Epoch 2/50
100/100 [==============================] - 8s 83ms/step - loss: 2.1432 - binary_crossentropy: 0.2683 - jaccard_coef_int: 0.4799 - binary_accuracy: 0.9282 - val_loss: 3.2238 - val_binary_crossentropy: 0.4808 - val_jaccard_coef_int: 0.3926 - val_binary_accuracy: 0.8715
Epoch 3/50
100/100 [==============================] - 8s 82ms/step - loss: 1.7212 - binary_crossentropy: 0.2503 - jaccard_coef_int: 0.5449 - binary_accuracy: 0.9411 - val_loss: 4.7746 - val_binary_crossentropy: 1.2457 - val_jaccard_coef_int: 0.2847 - val_binary_accuracy: 0.7886
Epoch 4/50
100/100 [==============================] - 8s 84ms/step - loss: 1.4181 - binary_crossentropy: 0.2199 - jaccard_coef_int: 0.5881 - binary_accuracy: 0.9

100/100 [==============================] - 8s 83ms/step - loss: 0.6530 - binary_crossentropy: 0.1659 - jaccard_coef_int: 0.7751 - binary_accuracy: 0.9757 - val_loss: 0.5561 - val_binary_crossentropy: 0.1344 - val_jaccard_coef_int: 0.7964 - val_binary_accuracy: 0.9800
Epoch 31/50
100/100 [==============================] - 9s 85ms/step - loss: 0.6185 - binary_crossentropy: 0.1532 - jaccard_coef_int: 0.7831 - binary_accuracy: 0.9772 - val_loss: 0.6110 - val_binary_crossentropy: 0.1510 - val_jaccard_coef_int: 0.7854 - val_binary_accuracy: 0.9776
Epoch 32/50
100/100 [==============================] - 8s 83ms/step - loss: 0.6217 - binary_crossentropy: 0.1568 - jaccard_coef_int: 0.7818 - binary_accuracy: 0.9771 - val_loss: 0.6976 - val_binary_crossentropy: 0.1673 - val_jaccard_coef_int: 0.7353 - val_binary_accuracy: 0.9758
Epoch 33/50
100/100 [==============================] - 8s 82ms/step - loss: 0.5887 - binary_crossentropy: 0.1448 - jaccard_coef_int: 0.7901 - binary_accuracy: 0.9785 - val_

Epoch 7/50
100/100 [==============================] - 11s 108ms/step - loss: 1.9146 - binary_crossentropy: 0.2164 - jaccard_coef_int: 0.3534 - binary_accuracy: 0.9495 - val_loss: 1.8518 - val_binary_crossentropy: 0.2341 - val_jaccard_coef_int: 0.3704 - val_binary_accuracy: 0.9491
Epoch 8/50
100/100 [==============================] - 11s 109ms/step - loss: 1.8375 - binary_crossentropy: 0.2234 - jaccard_coef_int: 0.3704 - binary_accuracy: 0.9508 - val_loss: 1.6693 - val_binary_crossentropy: 0.2227 - val_jaccard_coef_int: 0.4261 - val_binary_accuracy: 0.9519
Epoch 9/50
100/100 [==============================] - 11s 107ms/step - loss: 1.7164 - binary_crossentropy: 0.2203 - jaccard_coef_int: 0.3970 - binary_accuracy: 0.9535 - val_loss: 1.7162 - val_binary_crossentropy: 0.2491 - val_jaccard_coef_int: 0.4218 - val_binary_accuracy: 0.9497
Epoch 10/50
100/100 [==============================] - 11s 109ms/step - loss: 1.6412 - binary_crossentropy: 0.2214 - jaccard_coef_int: 0.4143 - binary_accura

...

Epoch 12/50
100/100 [==============================] - 11s 108ms/step - loss: 1.3992 - binary_crossentropy: 0.2233 - jaccard_coef_int: 0.5473 - binary_accuracy: 0.9534 - val_loss: 1.4137 - val_binary_crossentropy: 0.2052 - val_jaccard_coef_int: 0.5099 - val_binary_accuracy: 0.9561
Epoch 13/50
100/100 [==============================] - 11s 108ms/step - loss: 1.3203 - binary_crossentropy: 0.2099 - jaccard_coef_int: 0.5711 - binary_accuracy: 0.9557 - val_loss: 1.1007 - val_binary_crossentropy: 0.1422 - val_jaccard_coef_int: 0.6234 - val_binary_accuracy: 0.9648
Epoch 14/50
100/100 [==============================] - 11s 109ms/step - loss: 1.2675 - binary_crossentropy: 0.2013 - jaccard_coef_int: 0.5826 - binary_accuracy: 0.9576 - val_loss: 1.1732 - val_binary_crossentropy: 0.1748 - val_jaccard_coef_int: 0.5998 - val_binary_accuracy: 0.9617
Epoch 15/50
100/100 [==============================] - 11s 108ms/step - loss: 1.2184 - binary_crossentropy: 0.1933 - jaccard_coef_int: 0.5950 - binary_acc

100/100 [==============================] - 11s 108ms/step - loss: 0.6286 - binary_crossentropy: 0.1243 - jaccard_coef_int: 0.7678 - binary_accuracy: 0.9782 - val_loss: 0.4879 - val_binary_crossentropy: 0.0715 - val_jaccard_coef_int: 0.8101 - val_binary_accuracy: 0.9837
Epoch 42/50
100/100 [==============================] - 11s 109ms/step - loss: 0.5842 - binary_crossentropy: 0.1117 - jaccard_coef_int: 0.7843 - binary_accuracy: 0.9796 - val_loss: 0.4911 - val_binary_crossentropy: 0.0750 - val_jaccard_coef_int: 0.8045 - val_binary_accuracy: 0.9840
Epoch 43/50
100/100 [==============================] - 11s 109ms/step - loss: 0.5583 - binary_crossentropy: 0.1075 - jaccard_coef_int: 0.7937 - binary_accuracy: 0.9805 - val_loss: 0.4433 - val_binary_crossentropy: 0.0640 - val_jaccard_coef_int: 0.8218 - val_binary_accuracy: 0.9855
Epoch 44/50
100/100 [==============================] - 11s 109ms/step - loss: 0.5777 - binary_crossentropy: 0.1163 - jaccard_coef_int: 0.7877 - binary_accuracy: 0.979

Epoch 18/50
100/100 [==============================] - 9s 87ms/step - loss: 0.7448 - binary_crossentropy: 0.0884 - jaccard_coef_int: 0.6568 - binary_accuracy: 0.9816 - val_loss: 1.0565 - val_binary_crossentropy: 0.1080 - val_jaccard_coef_int: 0.4927 - val_binary_accuracy: 0.9803
Epoch 19/50
100/100 [==============================] - 9s 85ms/step - loss: 0.7127 - binary_crossentropy: 0.0838 - jaccard_coef_int: 0.6698 - binary_accuracy: 0.9824 - val_loss: 0.7930 - val_binary_crossentropy: 0.0775 - val_jaccard_coef_int: 0.6046 - val_binary_accuracy: 0.9843
Epoch 20/50
100/100 [==============================] - 8s 85ms/step - loss: 0.6726 - binary_crossentropy: 0.0783 - jaccard_coef_int: 0.6863 - binary_accuracy: 0.9833 - val_loss: 0.7634 - val_binary_crossentropy: 0.0732 - val_jaccard_coef_int: 0.6257 - val_binary_accuracy: 0.9842
Epoch 21/50
100/100 [==============================] - 8s 85ms/step - loss: 0.6627 - binary_crossentropy: 0.0779 - jaccard_coef_int: 0.6910 - binary_accuracy: 0

100/100 [==============================] - 9s 85ms/step - loss: 0.3728 - binary_crossentropy: 0.0480 - jaccard_coef_int: 0.8117 - binary_accuracy: 0.9906 - val_loss: 0.5244 - val_binary_crossentropy: 0.0641 - val_jaccard_coef_int: 0.7028 - val_binary_accuracy: 0.9900
Epoch 48/50
100/100 [==============================] - 9s 85ms/step - loss: 0.3714 - binary_crossentropy: 0.0483 - jaccard_coef_int: 0.8117 - binary_accuracy: 0.9907 - val_loss: 0.3542 - val_binary_crossentropy: 0.0332 - val_jaccard_coef_int: 0.7875 - val_binary_accuracy: 0.9935
Epoch 49/50
100/100 [==============================] - 8s 84ms/step - loss: 0.3583 - binary_crossentropy: 0.0457 - jaccard_coef_int: 0.8168 - binary_accuracy: 0.9911 - val_loss: 0.3711 - val_binary_crossentropy: 0.0355 - val_jaccard_coef_int: 0.7823 - val_binary_accuracy: 0.9930
Epoch 50/50
100/100 [==============================] - 9s 86ms/step - loss: 0.3668 - binary_crossentropy: 0.0480 - jaccard_coef_int: 0.8118 - binary_accuracy: 0.9909 - val_

100/100 [==============================] - 9s 86ms/step - loss: 0.8681 - binary_crossentropy: 0.1886 - jaccard_coef_int: 0.7432 - binary_accuracy: 0.9660 - val_loss: 0.9279 - val_binary_crossentropy: 0.2105 - val_jaccard_coef_int: 0.7411 - val_binary_accuracy: 0.9623
Epoch 25/50
100/100 [==============================] - 9s 85ms/step - loss: 0.8622 - binary_crossentropy: 0.1897 - jaccard_coef_int: 0.7453 - binary_accuracy: 0.9661 - val_loss: 0.9568 - val_binary_crossentropy: 0.2209 - val_jaccard_coef_int: 0.7376 - val_binary_accuracy: 0.9606
Epoch 26/50
100/100 [==============================] - 9s 86ms/step - loss: 0.8272 - binary_crossentropy: 0.1808 - jaccard_coef_int: 0.7536 - binary_accuracy: 0.9676 - val_loss: 0.7373 - val_binary_crossentropy: 0.1677 - val_jaccard_coef_int: 0.7908 - val_binary_accuracy: 0.9706
Epoch 27/50
100/100 [==============================] - 9s 86ms/step - loss: 0.7945 - binary_crossentropy: 0.1747 - jaccard_coef_int: 0.7637 - binary_accuracy: 0.9688 - val_